In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004038810729980469
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 13.02073611707761
200000 22.990852792059073
300000 25.416431020363323
400000 25.7516874737975
500000 24.95978830969132
600000 24.678210830143232
700000 24.38616296653564
800000 23.668729563333336
900000 25.98184725349371
1000000 11.230350349166088
1100000 23.740029886219
1200000 25.013748531506845
1300000 23.088331957169736
1400000 24.632617150144746
1500000 25.523825460507283
1600000 23.40707447233878
1700000 24.2298567117847
1800000 24.25341653116154
1900000 24.552059468789732
2000000 7.786104011566419
2100000 26.978203257465086
2200000 29.63283715698573
2300000 29.68212911934993
2400000 31.198036985151838
2500000 28.289858948827003
2600000 28.534052924012922
2700000 28.455176637539896
2800000 28.02618827962981
2900000 27.903725002230285
3000000 13.0371

29400000 23.31049317510274
29500000 24.452331513316373
29600000 10.235109728674141
29700000 22.111091094231526
29800000 23.949305552628058
29900000 22.63337798081705
30000000 22.799031427373052
30100000 20.976188773793933
30200000 20.593398470613078
30300000 9.956033412698737
30400000 19.39590678445237
30500000 20.698470486737186
30600000 21.528004590569427
30700000 21.42518383881123
30800000 22.765878140695733
30900000 22.657716468045866
31000000 17.342757580442935
31100000 15.779780294944494
31200000 22.760605705156507
31300000 23.025847481502375
31400000 23.642516096193514
31500000 18.79805863050338
31600000 14.149070323400458
31700000 22.86239596631417
31800000 22.248873479940546
31900000 23.149551799856834
32000000 21.29339079714571
32100000 20.0614059661856
32200000 18.496994926644593
32300000 9.746575993482976
32400000 20.062292511618217
32500000 20.5370921577999
32600000 21.63596490072674
32700000 22.088940854931028
32800000 22.60604615125059
32900000 22.93988190376278
33000000

59100000 10.271190564785206
59200000 17.32909700970965
59300000 17.717469059282816
59400000 17.91620367057985
59500000 17.103891154817568
59600000 7.535434731575497
59700000 17.61862900294167
59800000 17.679534788822856
59900000 17.75959988570877
60000000 17.392175100189338
60100000 9.293108658650086
60200000 18.878017515179295
60300000 20.54768337522823
60400000 20.52882588662466
60500000 20.75688237050426
60600000 22.128544282030873
60700000 20.80338746339154
60800000 20.2275998343429
60900000 19.55064818482699
61000000 20.085012467008763
61100000 7.990641144698747
61200000 18.09734095415795
61300000 17.987305390348308
61400000 18.948985440655417
61500000 17.0223069140604
61600000 9.42521553682171
61700000 18.24419481522084
61800000 18.317926939395264
61900000 17.737907583381034
62000000 14.21865568618438
62100000 14.181811573825668
62200000 20.177672879658857
62300000 19.984753161791218
62400000 19.86265615056075
62500000 20.887498187377854
62600000 20.13236603782667
62700000 19.594

89000000 15.660386155036441
89100000 15.68209205538066
89200000 9.25235031700602
89300000 15.848798116166144
89400000 16.027524010788255
89500000 16.31266813923051
89600000 17.118562597232103
89700000 16.99009387379208
89800000 17.481710563989104
89900000 18.020320752117055
90000000 17.54051198643493
90100000 16.854286591602953
90200000 16.352464765863534
90300000 16.356994336129137
90400000 14.678182147392999
90500000 15.627473586219912
90600000 8.640724675093265
90700000 15.081352647701
90800000 13.519046619363555
90900000 8.292442597240196
91000000 15.414702351420683
91100000 8.150661605338499
91200000 16.048416679818082
91300000 15.251050738514426
91400000 16.666225032082096
91500000 15.497951926154762
91600000 16.541980929080506
91700000 16.904619214794764
91800000 16.586888327794643
91900000 18.65504503306979
92000000 17.8042276539099
92100000 16.853395373709322
92200000 15.974716152144243
92300000 16.595310847450936
92400000 15.549833096943079
92500000 16.540054705428425
9260000

118100000 1.486374571769445
118200000 1.478059788635373
118300000 1.502165961980657
118400000 1.5348295041846374
118500000 1.4858700896662236
118600000 1.5654047827579696
118700000 1.540998668803789
118800000 1.564915581817939
118900000 1.5286186526575565
119000000 1.4887599015049637
119100000 1.4987959482012927
119200000 1.5069097066004844
119300000 1.5467584286844154
119400000 1.5126349152103404
119500000 1.4984219125729394
119600000 1.527732813406215
119700000 1.6466141444085518
119800000 1.5925349979962062
119900000 1.5845081892281407
120000000 1.5121547768532155
120100000 1.7169593553292375
120200000 1.7985192825394645
120300000 1.5063215604554474
120400000 1.5881170098728101
120500000 1.4785103312337537
120600000 1.4809704261934578
120700000 1.5263375214200152
120800000 1.5548942111496078
120900000 1.5694110308471623
121000000 1.568164284675533
121100000 1.5334748225949226
121200000 1.522121967122656
121300000 1.513165783691913
121400000 1.55534639361362
121500000 1.5636506116640

...

204700000 8.675140382898814
204800000 4.025165228612376
204900000 9.688754653273833
205000000 9.279994661373886
205100000 9.271018866006951
205200000 8.41100523675279
205300000 8.309294995444843
205400000 5.072062877548656
205500000 7.577118385716266
205600000 8.112338078614679
205700000 7.867909538793359
205800000 8.024954736001604
205900000 7.4105238730707255
206000000 7.488791256392919
206100000 2.890453702800161
206200000 7.672795340588413
206300000 7.450707319305736
206400000 7.828038161655968
206500000 7.895282663735566
206600000 8.124612672144213
206700000 7.898179461432479
206800000 4.2549051259685
206900000 8.209633261536126
207000000 8.200090446501955
207100000 8.576837745986763
207200000 8.341661888593721
207300000 7.202312431107849
207400000 5.4847799016772445
207500000 8.327296202001392
207600000 7.986810506301355
207700000 7.874316969413245
207800000 7.782523203653866
207900000 7.320710268939367
208000000 5.562805554557644
208100000 4.8016504146577095
208200000 7.30595445

234000000 6.6160649709249215
234100000 6.430098548876638
234200000 6.4722178948547295
234300000 4.174284943342519
234400000 5.451599915910566
234500000 3.2957743897385128
234600000 6.4053573647895234
234700000 6.223287388642389
234800000 6.330697668316377
234900000 6.365192945368088
235000000 5.981913218746126
235100000 5.941328169346766
235200000 6.170995943731332
235300000 6.131446369243903
235400000 2.2119683145134617
235500000 6.320578420746664
235600000 6.184694428356023
235700000 5.991729010870044
235800000 6.124626549073651
235900000 6.42560837559251
236000000 6.347420032794793
236100000 6.202878874482125
236200000 6.318994346747798
236300000 2.7343878100974375
236400000 4.046237116839786
236500000 5.7345575484984295
236600000 6.291224691330108
236700000 6.3622356271340985
236800000 6.523920616496932
236900000 6.164591313044184
237000000 5.989641715364993
237100000 6.04062076180326
237200000 6.275474656650647
237300000 4.549405418649129
237400000 4.206809648401493
237500000 6.32

262700000 0.4785021069022986
262800000 0.47447307855088117
262900000 1.178590295623349
263000000 2.3671103116155843
263100000 1.7486048083089172
263200000 1.9481192400669964
263300000 1.9292891026654804
263400000 1.7516639815148511
263500000 1.9155842105634795
263600000 1.8581407780348376
263700000 2.230244944963918
263800000 2.02823047876136
263900000 1.9711786846414245
264000000 1.7828990331298291
264100000 1.7470930089818888
264200000 1.7394596652704908
264300000 1.8618945335451682
264400000 1.9109490849525979
264500000 1.4857927834525082
264600000 0.7790481612771789
264700000 0.46109959187416366
264800000 0.651841739328536
264900000 1.518628558505541
265000000 2.1310908870651484
265100000 1.7982819752063879
265200000 1.6277763976908128
265300000 1.7193822069321256
265400000 1.7483953430631431
265500000 1.867886467465925
265600000 1.8448298183371497
265700000 2.1258171924622378
265800000 1.783109578866917
265900000 1.6702713909951443
266000000 1.6508952344618597
266100000 1.76188259

291200000 1.165629742851784
291300000 0.965401230939106
291400000 0.9940813260501371
291500000 1.0996147299779873
291600000 0.9759747737333794
291700000 0.879763881264925
291800000 1.1097225118821543
291900000 0.5551649109872461
292000000 0.23473482193636366
292100000 0.23527082057965334
292200000 0.5827335268692493
292300000 1.2387098449705423
292400000 0.889026673530572
292500000 1.024695256209004
292600000 1.030093101319576
292700000 0.920965871308189
292800000 1.0325916977591159
292900000 0.9805912118130935
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 1.0993700510216977
293100000 0.8589874536231578
293200000 0.8162242581684173
293300000 0.8632659207441469
293400000 0.8137423494179686
293500000 0.7208581827645587
293600000 0.6623271733171396
293700000 0.6844815587151586
293800000 0.7691914668299735
293900000 0.6364201871053558
294000000 0.6445581541450441
294100000 0.6275524698457413
294200000 0.721311503720351
294300000 0.6702804154133717
29440000

best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.025816354424699147
317400000 0.06643197801037033
317500000 0.03374009950738218
317600000 0.03789355281087558
317700000 0.051580344628171126
317800000 0.05015563622474935
317900000 0.053500606201104986
318000000 0.05217205424349904
318100000 0.03558395699599386
318200000 0.013373644367494187
318300000 0.010261910832230249
318400000 0.013772632423335977
318500000 0.017816151977569528
318600000 0.024684405533965427
318700000 0.02661500575884713
318800000 0.024641110587596892
318900000 0.02261892434582114
319000000 0.023263045025700993
319100000 0.022690926619591316
319200000 0.018514893200075627
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.013222776380005147
319400000 0.005277868473478158
319500000 0.00443425686019063
319600000 0.005212399597549439
best so far: 0
type: [1, 5, 1, 25, 1] 125
cases of this type: 78125
319700000 0.004447745524940888
best so far: 0
type: [1, 5, 